In [ ]:
import json
import math
from typing import List, Dict, Any

class DroneDeliveryOptimizer:
    def __init__(self, input_data: Dict[str, Any]):
        self.city_grid_size = input_data['city']['grid_size']
        self.drones = input_data['drones']['fleet']
        self.orders = input_data['orders']

    def calculate_distance(self, start: tuple, end: tuple) -> float:
        return abs(start[0] - end[0]) + abs(start[1] - end[1])

    def is_drone_suitable(self, drone: Dict[str, Any], order: Dict[str, Any]) -> bool:
        if not drone['available']:
            return False
        if order['package_weight'] > drone['max_payload']:
            return False
        order_location = (order['delivery_x'], order['delivery_y'])
        round_trip_distance = 2 * self.calculate_distance((0, 0), order_location)
        if round_trip_distance > drone['max_distance']:
            return False

        return True

    def optimize_deliveries(self) -> Dict[str, List[Dict[str, Any]]]:
        assignments = []
        used_orders = set()

        sorted_drones = sorted(
            [drone for drone in self.drones if drone['available']],
            key=lambda d: (d['max_payload'], d['speed']),
            reverse=True
        )

        for drone in sorted_drones:
            drone_orders = []
            drone_total_weight = 0
            drone_total_distance = 0

            remaining_orders = [
                order for order in self.orders
                if order['id'] not in used_orders
            ]
            remaining_orders.sort(key=lambda o: (o['deadline'], -o['package_weight']))

            for order in remaining_orders:
                if (self.is_drone_suitable(drone, order) and
                    drone_total_weight + order['package_weight'] <= drone['max_payload']):

                    order_location = (order['delivery_x'], order['delivery_y'])
                    order_distance = 2 * self.calculate_distance((0, 0), order_location)

                    drone_orders.append(order['id'])
                    used_orders.add(order['id'])
                    drone_total_weight += order['package_weight']
                    drone_total_distance = max(drone_total_distance, order_distance)

            if drone_orders:
                assignments.append({
                    "drone": drone['id'],
                    "orders": drone_orders,
                    "total_distance": round(drone_total_distance)
                })

        return {"assignments": assignments}

def main(input_file: str, output_file: str):
    with open(input_file, 'r') as f:
        input_data = json.load(f)
    optimizer = DroneDeliveryOptimizer(input_data)
    result = optimizer.optimize_deliveries()

    with open(output_file, 'w') as f:
        json.dump(result, f, indent=2)

    print("\nFinal Result:", json.dumps(result, indent=2))

if __name__ == "__main__":
    main('input.json', 'output.json')


Final Result: {
  "assignments": [
    {
      "drone": "D2",
      "orders": [
        "O1",
        "O3",
        "O2"
      ],
      "total_distance": 16
    },
    {
      "drone": "D1",
      "orders": [
        "O4"
      ],
      "total_distance": 50
    }
  ]
}
